In [ ]:
import numpy as np
import pandas as pd
import numba as nb

string_limit = 850

anno = pd.read_csv("./annotation.csv")

In [ ]:
sample = anno[anno["time"] != 'Baseline']["geo"]
ctrl = anno[anno["time"] == 'Baseline']["geo"]

In [ ]:
fpkm = pd.read_csv("./fpkm2.csv")
weight = pd.read_csv("./weight.csv")
id = pd.read_csv("./id2.csv")

In [ ]:
stringdb = pd.read_csv("StringDB_link.csv", delimiter=" ")
stringdb["protein1"] = stringdb["protein1"].str.slice(start=5)
stringdb["protein2"] = stringdb["protein2"].str.slice(start=5)
stringdb = stringdb[stringdb["combined_score"] >= string_limit]
stringdb = stringdb.reset_index(drop=True)

dict1 = pd.read_csv("./id2.csv")
length = len(np.unique(dict1["symbol"].values))
fpkm = pd.read_csv("./fpkm2.csv")

d = {dict1["ensp"][i]: int(dict1["id"][i]) for i in range(len(dict1))}
d2 = {int(dict1["id"][i]): dict1["symbol"][i] for i in range(len(dict1))}
dw = {weight['id'][i]: weight['Length'][i] for i in range(len(weight))}
string_bool = np.zeros([length, length])


for i in range(len(stringdb)):
    if (stringdb["protein1"][i] in d) & (stringdb["protein2"][i] in d):
        string_bool[d[stringdb["protein1"][i]], d[stringdb["protein2"][i]]] = 1

np.fill_diagonal(string_bool, 0)
string_bool = string_bool == 1

origin_frame = fpkm[ctrl].to_numpy()
origin_sd = np.std(origin_frame, ddof=1, axis=1)

num = len(origin_frame)

In [ ]:
origin_pc = np.corrcoef(origin_frame)
origin_pc = np.abs(origin_pc)
origin_pc = origin_pc * string_bool

In [ ]:
edge_origin_list = []

idlist1 = []
idlist2 = []
for i in range(num):
    for j in range(i - 1):
        if string_bool[i, j] != 0:
            edge_origin_list.append([d2[i], d2[j]])
            idlist1.append(i)
            idlist2.append(j)

edge_origin_entropy = np.zeros((len(idlist1)), dtype=np.float64)
edge_origin_sd = np.zeros((len(idlist1)), dtype=np.float64)
idlist1 = nb.typed.List(idlist1)
idlist2 = nb.typed.List(idlist2)


In [ ]:
weight = pd.read_csv("./weight.csv")

In [ ]:
dw = nb.typed.List(weight['weight'])

In [ ]:
# Entropy calculation
@nb.njit(parallel=False)
def entropy(pc, sd, idlist1, idlist2, edge_entropy, edge_sd, dw):
    for i in range(len(idlist1)):
        p1 = idlist1[i]
        p2 = idlist2[i]

        left_not_zero = np.where(pc[p1] != 0)[0]
        right_not_zero = np.where(pc[p2] != 0)[0]

        left_not_zero = np.delete(left_not_zero, np.where(left_not_zero == p2)[0])
        right_not_zero = np.delete(right_not_zero, np.where(right_not_zero == p1)[0])

        len_left_not_zero = len(left_not_zero)
        len_right_not_zero = len(right_not_zero)

        if len_left_not_zero < 2 and len_right_not_zero < 2:
            edge_entropy[i] = 0
            edge_sd[i] = 0
            continue
        if len_left_not_zero < 2 and len_right_not_zero > 1:
            right_prob = (pc[p2][right_not_zero]) / np.sum((pc[p2][right_not_zero]))
            right_entropy = -np.sum(right_prob * np.log2(right_prob)) / np.log2(
                len_right_not_zero
            )
            entropy = (
                right_entropy
                / (dw[p1] + dw[p2])
                * dw[p2]
            )
            edge_entropy[i] = entropy
            edge_sd[i] = (sd[p1] * dw[p1] + sd[p2] * dw[p2]) / (
                dw[p1] + dw[p2]
            )
            continue
        if len_left_not_zero > 1 and len_right_not_zero < 2:
            left_prob = (pc[p1][left_not_zero]) / np.sum((pc[p1][left_not_zero]))
            left_entropy = -np.sum(left_prob * np.log2(left_prob)) / np.log2(
                len_left_not_zero
            )
            entropy = (
                left_entropy
                / (dw[p1] + dw[p2])
                * dw[p1]
            )
            edge_entropy[i] = entropy
            edge_sd[i] = (sd[p1] * dw[p1] + sd[p2] * dw[p2]) / (
                dw[p1] + dw[p2]
            )
            continue

        edge_sd[i] = (sd[p1] * dw[p1] + sd[p2] * dw[p2]) / (
                dw[p1] + dw[p2]
        )
        left_prob = (pc[p1][left_not_zero]) / np.sum((pc[p1][left_not_zero]))
        right_prob = (pc[p2][right_not_zero]) / np.sum((pc[p2][right_not_zero]))
        left_entropy = -np.sum(left_prob * np.log2(left_prob)) / np.log2(
            len_left_not_zero
        )
        right_entropy = -np.sum(right_prob * np.log2(right_prob)) / np.log2(
            len_right_not_zero
        )
        entropy = (
            right_entropy * dw[p1] + left_entropy * dw[p2]
        ) / (dw[p1] + dw[p2])
        edge_entropy[i] = entropy

In [ ]:
entropy(origin_pc, origin_sd, idlist1, idlist2, edge_origin_entropy, edge_origin_sd, dw)

In [ ]:
append_frame2 = pd.read_csv("./fpkm2.csv",index_col=0)
append_len = np.shape(append_frame2)[1]
id_len = len(idlist1)
landscape2 = pd.DataFrame()


In [ ]:
for k in append_frame2.columns:
    print(k, end="\n")

    append_frame3 = np.column_stack((origin_frame, append_frame2[k]))
    append_pc = np.corrcoef(append_frame3)
    append_pc = np.abs(append_pc)
    append_pc = append_pc * string_bool
    append_sd = np.std(append_frame3, ddof=1, axis=1)
    edge_append_entropy = np.zeros((len(idlist1)), dtype=np.float64)
    edge_append_sd = np.zeros((len(idlist1)), dtype=np.float64)

    entropy(append_pc, append_sd, idlist1, idlist2, edge_append_entropy, edge_append_sd, dw)

    edge_append_entropy = np.abs(edge_append_entropy - edge_origin_entropy)
    edge_append_sd = np.abs(edge_append_sd - edge_origin_sd)

    landscape_pros = pd.DataFrame(edge_append_sd * edge_append_entropy)
    landscape_pros.columns = [k]
    landscape2 = pd.concat([landscape2, landscape_pros], axis=1)

In [ ]:
landscape2 = pd.concat([pd.DataFrame(edge_origin_list, columns=['node1', 'node2']), landscape2], axis=1)
landscape2 = landscape2.fillna(0)
landscape2.to_csv("edge_entropy_logM-1_" + str(string_limit) + "_weight.csv", index=False)